# Semantic Scholar Data Import Notebook

This notebook demonstrates how to import and work with data from the Semantic Scholar Academic Graph API. Semantic Scholar provides comprehensive academic paper data with advanced features like influence metrics, citation contexts, and AI-powered paper recommendations.

## Semantic Scholar API Overview
- **Base URL**: https://api.semanticscholar.org/graph/v1/
- **Authentication**: API key required for higher rate limits
- **Rate Limiting**: 100 requests/5 minutes (free), 1000 requests/5 minutes (with API key)
- **Coverage**: 200M+ papers across all fields of science

## What we'll cover:
1. API setup and authentication
2. Fetching papers with detailed metadata
3. Author information and collaboration networks
4. CitedBy analysis and contexts
5. Recommendations and related papers
6. Advanced search and filtering
7. Data processing and export

## 1. Import Required Libraries

In [ ]:
# Essential libraries for API interaction and data handling
import requests
import pandas as pd
import json
import time
from typing import Dict, List, Optional, Any, Union
from urllib.parse import urljoin, urlencode, quote
import warnings
warnings.filterwarnings('ignore')

# For data visualization and analysis
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    print("Visualization libraries loaded successfully")
except ImportError:
    print("Visualization libraries not available - install matplotlib, seaborn, numpy for plots")

# For text processing
try:
    import re
    from collections import Counter, defaultdict
    print("Text processing libraries loaded")
except ImportError:
    print("Some text processing libraries not available")

print("All core libraries imported successfully!")

## 2. API Configuration and Authentication

In [ ]:
# Semantic Scholar API Configuration
class SemanticScholarConfig:
    """Configuration class for Semantic Scholar API"""
    
    BASE_URL = "https://api.semanticscholar.org/graph/v1"
    
    # Get your API key from: https://www.semanticscholar.org/product/api#api-key
    # Without API key: 100 requests per 5 minutes
    # With API key: 1000 requests per 5 minutes
    API_KEY = None  # Set your API key here or use environment variable
    
    # Request headers
    @property
    def headers(self):
        headers = {
            "User-Agent": "KnowledgeFabric/1.0 (mailto:your-email@domain.com)",
            "Accept": "application/json"
        }
        if self.API_KEY:
            headers["x-pipelines-key"] = self.API_KEY
        return headers
    
    # Rate limiting based on API key availability
    @property
    def request_delay(self):
        if self.API_KEY:
            return 0.31  # ~3 requests per second (1000/5min = 3.33/sec)
        else:
            return 3.1   # ~0.33 requests per second (100/5min = 0.33/sec)
    
    # Common field sets for different queries
    PAPER_FIELDS = [
        "paperId", "externalIds", "title", "abstract", "venue", "year", 
        "referenceCount", "citationCount", "influentialCitationCount",
        "isOpenAccess", "openAccessPdf", "fieldsOfStudy", "s2FieldsOfStudy",
        "authors", "citations", "references", "embedding", "tldr"
    ]
    
    AUTHOR_FIELDS = [
        "authorId", "externalIds", "name", "aliases", "affiliations",
        "homepage", "paperCount", "citationCount", "hIndex", "papers"
    ]

# Initialize configuration
config = SemanticScholarConfig()

# Check if API key is set (you can set it here or via environment variable)
import os
if not config.API_KEY:
    config.API_KEY = os.getenv('SEMANTIC_SCHOLAR_API_KEY')

print(f"Semantic Scholar API Base URL: {config.BASE_URL}")
print(f"API Key configured: {'Yes' if config.API_KEY else 'No (using free tier)'}")
print(f"Request delay: {config.request_delay:.2f} seconds")
print(f"Rate limit: {'1000 req/5min' if config.API_KEY else '100 req/5min'}")

## 3. Core API Request Functions

In [ ]:
def make_s2_request(endpoint: str, params: Dict = None, delay: bool = True) -> Dict:
    """
    Make a request to the Semantic Scholar API with error handling and rate limiting.
    
    Args:
        endpoint: API endpoint (e.g., 'paper/search', 'author/search')
        params: Query parameters
        delay: Whether to add delay for rate limiting
    
    Returns:
        JSON response as dictionary
    """
    if params is None:
        params = {}
    
    # Build URL
    url = urljoin(config.BASE_URL, endpoint)
    
    try:
        # Rate limiting
        if delay:
            time.sleep(config.request_delay)
        
        # Make request
        response = requests.get(url, headers=config.headers, params=params)
        response.raise_for_status()
        
        return response.json()
        
    except requests.exceptions.RequestException as e:
        print(f"Error making request to {url}: {e}")
        if hasattr(e.response, 'status_code'):
            print(f"Status code: {e.response.status_code}")
            if e.response.status_code == 429:
                print("Rate limit exceeded. Consider adding an API key or increasing delays.")
        return None

def test_s2_connection():
    """Test the Semantic Scholar API connection"""
    print("Testing Semantic Scholar API connection...")
    
    # Test with a simple paper search
    response = make_s2_request("paper/search", {
        "query": "attention is all you need",
        "limit": 1,
        "fields": "title,year,authors,citationCount"
    })
    
    if response and "data" in response:
        paper = response["data"][0] if response["data"] else None
        if paper:
            print("✅ API connection successful!")
            print(f"Test paper: {paper.get('title', 'Unknown')}")
            print(f"Citations: {paper.get('citationCount', 0):,}")
            return True
        else:
            print("❌ API connection failed - no data returned")
            return False
    else:
        print("❌ API connection failed!")
        return False

# Test the connection
test_s2_connection()

## 4. Paper Search and Retrieval

In [ ]:
def search_papers(query: str, limit: int = 100, fields: List[str] = None, 
                  year_filter: str = None, venue_filter: str = None,
                  field_of_study: str = None) -> pd.DataFrame:
    """
    Search for papers using Semantic Scholar API.
    
    Args:
        query: Search query string
        limit: Maximum number of results
        fields: List of fields to retrieve
        year_filter: Year range filter (e.g., "2020-2023")
        venue_filter: Venue name filter
        field_of_study: Field of study filter
    
    Returns:
        DataFrame with paper data
    """
    if fields is None:
        fields = ["paperId", "title", "abstract", "year", "authors", "venue",
                 "citationCount", "influentialCitationCount", "isOpenAccess", 
                 "fieldsOfStudy", "tldr"]
    
    papers_data = []
    offset = 0
    batch_size = min(100, limit)  # API max is 100 per request
    
    print(f"Searching for papers: '{query}' (limit: {limit})")
    
    while offset < limit:
        current_batch = min(batch_size, limit - offset)
        
        params = {
            "query": query,
            "limit": current_batch,
            "offset": offset,
            "fields": ",".join(fields)
        }
        
        # Add optional filters
        if year_filter:
            params["year"] = year_filter
        if venue_filter:
            params["venue"] = venue_filter
        if field_of_study:
            params["fieldsOfStudy"] = field_of_study
        
        response = make_s2_request("paper/search", params)
        
        if not response or "data" not in response:
            print(f"Failed to fetch papers at offset {offset}")
            break
        
        batch_papers = response["data"]
        if not batch_papers:
            print(f"No more papers found at offset {offset}")
            break
        
        # Process each paper
        for paper in batch_papers:
            paper_data = {
                "paper_id": paper.get("paperId"),
                "title": paper.get("title"),
                "abstract": paper.get("abstract"),
                "year": paper.get("year"),
                "venue": paper.get("venue"),
                "citation_count": paper.get("citationCount", 0),
                "influential_citation_count": paper.get("influentialCitationCount", 0),
                "is_open_access": paper.get("isOpenAccess", False),
                "fields_of_study": paper.get("fieldsOfStudy", []),
                "tldr": paper.get("tldr", {}).get("text") if paper.get("tldr") else None,
                "author_count": len(paper.get("authors", [])),
                "author_names": [auth.get("name") for auth in paper.get("authors", []) if auth.get("name")],
                "first_author": paper.get("authors", [{}])[0].get("name") if paper.get("authors") else None
            }
            papers_data.append(paper_data)
        
        offset += current_batch
        print(f"Fetched {len(papers_data)} papers so far...")
        
        # Check if we've reached the total available
        total = response.get("total", 0)
        if total > 0 and len(papers_data) >= total:
            print(f"Reached total available papers: {total}")
            break
    
    df = pd.DataFrame(papers_data)
    print(f"Search completed. Retrieved {len(df)} papers.")
    
    return df

def get_paper_details(paper_id: str, include_citations: bool = False, 
                     include_references: bool = False) -> Dict:
    """
    Get detailed information for a specific paper.
    
    Args:
        paper_id: Semantic Scholar paper ID or DOI
        include_citations: Whether to include citation data
        include_references: Whether to include reference data
    
    Returns:
        Detailed paper information
    """
    fields = ["paperId", "externalIds", "title", "abstract", "venue", "year",
             "referenceCount", "citationCount", "influentialCitationCount",
             "isOpenAccess", "openAccessPdf", "fieldsOfStudy", "s2FieldsOfStudy",
             "authors", "embedding", "tldr"]
    
    if include_citations:
        fields.append("citations")
    if include_references:
        fields.append("references")
    
    params = {"fields": ",".join(fields)}
    
    response = make_s2_request(f"paper/{paper_id}", params)
    return response

# Example: Search for recent transformer papers
print("Example 1: Searching for transformer papers...")
transformer_papers = search_papers(
    query="transformer attention mechanism",
    limit=50,
    year_filter="2020-2024",
    field_of_study="Computer Science"
)

print(f"\nFound {len(transformer_papers)} transformer papers:")
print(transformer_papers[["title", "year", "citation_count", "venue", "is_open_access"]].head())

## 5. Author Information and Networks

In [ ]:
def search_authors(query: str, limit: int = 50) -> pd.DataFrame:
    """
    Search for authors using Semantic Scholar API.
    
    Args:
        query: Author name or affiliation search query
        limit: Maximum number of results
    
    Returns:
        DataFrame with author data
    """
    authors_data = []
    offset = 0
    batch_size = min(100, limit)
    
    print(f"Searching for authors: '{query}' (limit: {limit})")
    
    while offset < limit:
        current_batch = min(batch_size, limit - offset)
        
        params = {
            "query": query,
            "limit": current_batch,
            "offset": offset,
            "fields": ",".join(config.AUTHOR_FIELDS)
        }
        
        response = make_s2_request("author/search", params)
        
        if not response or "data" not in response:
            print(f"Failed to fetch authors at offset {offset}")
            break
        
        batch_authors = response["data"]
        if not batch_authors:
            break
        
        # Process each author
        for author in batch_authors:
            # Get current affiliation
            affiliations = author.get("affiliations", [])
            current_affiliation = affiliations[0] if affiliations else None
            
            author_data = {
                "author_id": author.get("authorId"),
                "name": author.get("name"),
                "aliases": author.get("aliases", []),
                "paper_count": author.get("paperCount", 0),
                "citation_count": author.get("citationCount", 0),
                "h_index": author.get("hIndex", 0),
                "homepage": author.get("homepage"),
                "current_affiliation": current_affiliation,
                "external_ids": author.get("externalIds", {}),
                "orcid": author.get("externalIds", {}).get("ORCID"),
                "google_scholar": author.get("externalIds", {}).get("GoogleScholar")
            }
            authors_data.append(author_data)
        
        offset += current_batch
        print(f"Fetched {len(authors_data)} authors so far...")
    
    df = pd.DataFrame(authors_data)
    print(f"Author search completed. Retrieved {len(df)} authors.")
    
    return df

def get_author_details(author_id: str, include_papers: bool = False) -> Dict:
    """
    Get detailed information for a specific author.
    
    Args:
        author_id: Semantic Scholar author ID
        include_papers: Whether to include author's papers
    
    Returns:
        Detailed author information
    """
    fields = config.AUTHOR_FIELDS.copy()
    if not include_papers and "papers" in fields:
        fields.remove("papers")
    
    params = {"fields": ",".join(fields)}
    
    response = make_s2_request(f"author/{author_id}", params)
    return response

def analyze_collaboration_network(papers_df: pd.DataFrame) -> pd.DataFrame:
    """
    Analyze collaboration networks from papers DataFrame.
    
    Args:
        papers_df: DataFrame containing papers with author information
    
    Returns:
        DataFrame with collaboration statistics
    """
    collaborations = []
    
    for _, paper in papers_df.iterrows():
        authors = paper.get("author_names", [])
        if len(authors) > 1:
            # Count collaborations between all pairs of authors
            for i, author1 in enumerate(authors):
                for author2 in authors[i+1:]:
                    collaborations.append({
                        "author1": author1,
                        "author2": author2,
                        "paper_title": paper.get("title"),
                        "year": paper.get("year"),
                        "citation_count": paper.get("citation_count", 0)
                    })
    
    if not collaborations:
        return pd.DataFrame()
    
    collab_df = pd.DataFrame(collaborations)
    
    # Aggregate collaboration statistics
    collab_stats = collab_df.groupby(["author1", "author2"]).agg({
        "paper_title": "count",
        "citation_count": ["sum", "mean"]
    }).round(2)
    
    collab_stats.columns = ["collaboration_count", "total_citations", "avg_citations"]
    collab_stats = collab_stats.reset_index()
    
    return collab_stats.sort_values("collaboration_count", ascending=False)

# Example: Search for AI researchers
print("Example 2: Searching for AI researchers...")
ai_researchers = search_authors("artificial intelligence", limit=25)

print(f"\nFound {len(ai_researchers)} AI researchers:")
print(ai_researchers[["name", "paper_count", "citation_count", "h_index", "current_affiliation"]].head())

## 6. CitedBy Analysis and Contexts

In [ ]:
def get_paper_citations(paper_id: str, limit: int = 100) -> pd.DataFrame:
    """
    Get citations for a specific paper with citation contexts.
    
    Args:
        paper_id: Semantic Scholar paper ID
        limit: Maximum number of citations to retrieve
    
    Returns:
        DataFrame with citation information
    """
    citations_data = []
    offset = 0
    batch_size = min(1000, limit)  # API allows large batches for citations
    
    print(f"Fetching citations for paper {paper_id}...")
    
    while offset < limit:
        params = {
            "offset": offset,
            "limit": min(batch_size, limit - offset),
            "fields": "paperId,title,year,authors,citationCount,contexts"
        }
        
        response = make_s2_request(f"paper/{paper_id}/citations", params)
        
        if not response or "data" not in response:
            print(f"Failed to fetch citations at offset {offset}")
            break
        
        batch_citations = response["data"]
        if not batch_citations:
            break
        
        # Process each citation
        for citation in batch_citations:
            citing_paper = citation.get("citingPaper", {})
            contexts = citation.get("contexts", [])
            
            citation_data = {
                "citing_paper_id": citing_paper.get("paperId"),
                "citing_title": citing_paper.get("title"),
                "citing_year": citing_paper.get("year"),
                "citing_citation_count": citing_paper.get("citationCount", 0),
                "citing_authors": [auth.get("name") for auth in citing_paper.get("authors", [])],
                "citation_contexts": [ctx for ctx in contexts if ctx],
                "context_count": len([ctx for ctx in contexts if ctx])
            }
            citations_data.append(citation_data)
        
        offset += len(batch_citations)
        print(f"Fetched {len(citations_data)} citations so far...")
        
        # Check if we've reached the end
        if len(batch_citations) < batch_size:
            break
    
    df = pd.DataFrame(citations_data)
    print(f"CitedBy fetch completed. Retrieved {len(df)} citations.")
    
    return df

def analyze_citation_contexts(citations_df: pd.DataFrame) -> Dict[str, Any]:
    """
    Analyze citation contexts to understand how papers are being cited.
    
    Args:
        citations_df: DataFrame with citation context data
    
    Returns:
        Dictionary with citation context analysis
    """
    if citations_df.empty:
        return {}
    
    # Basic statistics
    total_citations = len(citations_df)
    citations_with_context = citations_df[citations_df["context_count"] > 0]
    context_rate = len(citations_with_context) / total_citations if total_citations > 0 else 0
    
    # Temporal analysis
    yearly_citations = citations_df.groupby("citing_year").size().to_dict()
    
    # Context analysis
    all_contexts = []
    for contexts in citations_df["citation_contexts"]:
        all_contexts.extend(contexts)
    
    # Simple keyword analysis from contexts
    context_keywords = Counter()
    for context in all_contexts:
        if context and len(context) > 10:  # Filter very short contexts
            # Simple keyword extraction (you could use more sophisticated NLP)
            words = re.findall(r'\b[a-zA-Z]{4,}\b', context.lower())
            context_keywords.update(words)
    
    analysis = {
        "total_citations": total_citations,
        "citations_with_context": len(citations_with_context),
        "context_coverage_rate": context_rate,
        "yearly_distribution": yearly_citations,
        "avg_contexts_per_citation": citations_df["context_count"].mean(),
        "top_context_keywords": dict(context_keywords.most_common(10)),
        "citation_years_range": (citations_df["citing_year"].min(), citations_df["citing_year"].max()) if total_citations > 0 else None
    }
    
    return analysis

def get_paper_references(paper_id: str, limit: int = 100) -> pd.DataFrame:
    """
    Get references for a specific paper.
    
    Args:
        paper_id: Semantic Scholar paper ID
        limit: Maximum number of references to retrieve
    
    Returns:
        DataFrame with reference information
    """
    references_data = []
    offset = 0
    batch_size = min(1000, limit)
    
    print(f"Fetching references for paper {paper_id}...")
    
    while offset < limit:
        params = {
            "offset": offset,
            "limit": min(batch_size, limit - offset),
            "fields": "paperId,title,year,authors,citationCount,venue"
        }
        
        response = make_s2_request(f"paper/{paper_id}/references", params)
        
        if not response or "data" not in response:
            break
        
        batch_references = response["data"]
        if not batch_references:
            break
        
        # Process each reference
        for reference in batch_references:
            cited_paper = reference.get("citedPaper", {})
            
            ref_data = {
                "referenced_paper_id": cited_paper.get("paperId"),
                "referenced_title": cited_paper.get("title"),
                "referenced_year": cited_paper.get("year"),
                "referenced_citation_count": cited_paper.get("citationCount", 0),
                "referenced_venue": cited_paper.get("venue"),
                "referenced_authors": [auth.get("name") for auth in cited_paper.get("authors", [])]
            }
            references_data.append(ref_data)
        
        offset += len(batch_references)
        
        if len(batch_references) < batch_size:
            break
    
    df = pd.DataFrame(references_data)
    print(f"Reference fetch completed. Retrieved {len(df)} references.")
    
    return df

# Example: Analyze citations for a highly cited paper
if 'transformer_papers' in locals() and len(transformer_papers) > 0:
    # Get the most cited paper from our search
    most_cited = transformer_papers.loc[transformer_papers["citation_count"].idxmax()]
    print(f"\nExample 3: Analyzing citations for '{most_cited['title']}'")
    print(f"Paper has {most_cited['citation_count']:,} citations")
    
    # Get some citations (limit to 50 for demo)
    citations = get_paper_citations(most_cited["paper_id"], limit=50)
    
    if not citations.empty:
        print(f"\nSample of {len(citations)} citations:")
        print(citations[["citing_title", "citing_year", "citing_citation_count", "context_count"]].head())
        
        # Analyze citation contexts
        context_analysis = analyze_citation_contexts(citations)
        print(f"\nCitedBy Context Analysis:")
        print(f"- Citations with context: {context_analysis.get('citations_with_context', 0)}")
        print(f"- Context coverage: {context_analysis.get('context_coverage_rate', 0):.2%}")
        print(f"- Avg contexts per citation: {context_analysis.get('avg_contexts_per_citation', 0):.1f}")
else:
    print("No transformer papers available for citation analysis. Run the paper search cell first.")

## 7. Recommendations and Related Papers

In [ ]:
def get_recommendations(paper_id: str, limit: int = 20) -> pd.DataFrame:
    """
    Get paper recommendations based on a seed paper.
    
    Args:
        paper_id: Semantic Scholar paper ID to base recommendations on
        limit: Maximum number of recommendations
    
    Returns:
        DataFrame with recommended papers
    """
    params = {
        "fields": "paperId,title,year,authors,abstract,citationCount,influentialCitationCount,venue",
        "limit": limit
    }
    
    response = make_s2_request(f"recommendations/v1/papers/forpaper/{paper_id}", params)
    
    if not response or "recommendedPapers" not in response:
        print("Failed to fetch recommendations")
        return pd.DataFrame()
    
    recommendations_data = []
    
    for paper in response["recommendedPapers"]:
        rec_data = {
            "paper_id": paper.get("paperId"),
            "title": paper.get("title"),
            "year": paper.get("year"),
            "venue": paper.get("venue"),
            "citation_count": paper.get("citationCount", 0),
            "influential_citation_count": paper.get("influentialCitationCount", 0),
            "author_count": len(paper.get("authors", [])),
            "authors": [auth.get("name") for auth in paper.get("authors", [])],
            "has_abstract": bool(paper.get("abstract"))
        }
        recommendations_data.append(rec_data)
    
    df = pd.DataFrame(recommendations_data)
    print(f"Retrieved {len(df)} recommendations")
    
    return df

def batch_search_by_ids(paper_ids: List[str], fields: List[str] = None) -> pd.DataFrame:
    """
    Retrieve multiple papers by their IDs in batch.
    
    Args:
        paper_ids: List of paper IDs
        fields: Fields to retrieve
    
    Returns:
        DataFrame with paper information
    """
    if fields is None:
        fields = ["paperId", "title", "year", "authors", "citationCount", "venue"]
    
    papers_data = []
    
    # Process in batches (API supports multiple IDs in one request)
    batch_size = 500  # Semantic Scholar allows large batches
    
    for i in range(0, len(paper_ids), batch_size):
        batch_ids = paper_ids[i:i+batch_size]
        
        # Join IDs with commas
        ids_param = ",".join(batch_ids)
        
        params = {"fields": ",".join(fields)}
        
        response = make_s2_request(f"paper/batch", {**params, "ids": ids_param})
        
        if response:
            for paper in response:
                if paper:  # Some papers might not be found
                    paper_data = {
                        "paper_id": paper.get("paperId"),
                        "title": paper.get("title"),
                        "year": paper.get("year"),
                        "venue": paper.get("venue"),
                        "citation_count": paper.get("citationCount", 0),
                        "author_count": len(paper.get("authors", [])),
                        "authors": [auth.get("name") for auth in paper.get("authors", [])]
                    }
                    papers_data.append(paper_data)
        
        print(f"Processed batch {i//batch_size + 1}/{(len(paper_ids) + batch_size - 1)//batch_size}")
    
    return pd.DataFrame(papers_data)

def find_influential_papers(query: str, min_influential_citations: int = 50) -> pd.DataFrame:
    """
    Find highly influential papers based on influential citation count.
    
    Args:
        query: Search query
        min_influential_citations: Minimum influential citation count
    
    Returns:
        DataFrame with influential papers
    """
    # Search for papers
    papers = search_papers(query, limit=200)
    
    if papers.empty:
        return papers
    
    # Filter by influential citation count
    influential = papers[papers["influential_citation_count"] >= min_influential_citations]
    
    # Sort by influential citation count
    influential = influential.sort_values("influential_citation_count", ascending=False)
    
    print(f"Found {len(influential)} influential papers (≥{min_influential_citations} influential citations)")
    
    return influential

def analyze_research_trends(papers_df: pd.DataFrame, time_window: int = 5) -> Dict[str, Any]:
    """
    Analyze research trends from papers DataFrame.
    
    Args:
        papers_df: DataFrame containing papers with year information
        time_window: Years to consider for recent trend analysis
    
    Returns:
        Dictionary with trend analysis
    """
    if papers_df.empty or "year" not in papers_df.columns:
        return {}
    
    # Filter valid years
    valid_papers = papers_df[papers_df["year"].notna() & (papers_df["year"] > 1950)]
    
    if valid_papers.empty:
        return {}
    
    current_year = 2024  # or datetime.now().year
    recent_cutoff = current_year - time_window
    
    # Temporal trends
    yearly_counts = valid_papers.groupby("year").size().to_dict()
    recent_papers = valid_papers[valid_papers["year"] >= recent_cutoff]
    
    # CitedBy trends
    citation_stats = {
        "total_papers": len(valid_papers),
        "recent_papers": len(recent_papers),
        "avg_citations_all": valid_papers["citation_count"].mean(),
        "avg_citations_recent": recent_papers["citation_count"].mean() if not recent_papers.empty else 0,
        "yearly_publication_count": yearly_counts,
        "top_cited_recent": recent_papers.nlargest(5, "citation_count")[["title", "year", "citation_count"]].to_dict("records") if not recent_papers.empty else []
    }
    
    # Venue analysis
    if "venue" in valid_papers.columns:
        venue_counts = valid_papers["venue"].value_counts().head(10).to_dict()
        citation_stats["top_venues"] = venue_counts
    
    return citation_stats

# Example: Get recommendations for a transformer paper
if 'transformer_papers' in locals() and len(transformer_papers) > 0:
    sample_paper = transformer_papers.iloc[0]
    print(f"\nExample 4: Getting recommendations for '{sample_paper['title']}'")
    
    recommendations = get_recommendations(sample_paper["paper_id"], limit=10)
    
    if not recommendations.empty:
        print(f"\nTop 5 recommendations:")
        print(recommendations[["title", "year", "citation_count", "venue"]].head())
        
        # Analyze trends in transformer research
        print(f"\nTransformer Research Trends:")
        trends = analyze_research_trends(transformer_papers)
        print(f"- Total papers: {trends.get('total_papers', 0)}")
        print(f"- Recent papers (last 5 years): {trends.get('recent_papers', 0)}")
        print(f"- Average citations: {trends.get('avg_citations_all', 0):.1f}")
else:
    print("No transformer papers available for recommendations. Run the paper search cell first.")

## 8. Advanced Data Processing

In [ ]:
def process_semantic_scholar_data(papers_df: pd.DataFrame) -> pd.DataFrame:
    """
    Advanced processing of Semantic Scholar paper data.
    
    Args:
        papers_df: Raw papers DataFrame
        
    Returns:
        Processed and enriched DataFrame
    """
    df = papers_df.copy()
    
    print(f"Processing {len(df)} papers...")
    
    # Data cleaning
    df["title"] = df["title"].fillna("").str.strip()
    df["abstract"] = df["abstract"].fillna("")
    
    # Year validation
    current_year = 2024
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    df["year_valid"] = (df["year"] >= 1950) & (df["year"] <= current_year)
    
    # CitedBy metrics
    df["citation_count"] = pd.to_numeric(df["citation_count"], errors="coerce").fillna(0)
    df["influential_citation_count"] = pd.to_numeric(df["influential_citation_count"], errors="coerce").fillna(0)
    
    # Calculate influence ratio (what % of citations are influential)
    df["influence_ratio"] = np.where(
        df["citation_count"] > 0,
        df["influential_citation_count"] / df["citation_count"],
        0
    )
    
    # Recency scoring (higher score for recent papers)
    df["recency_score"] = np.where(
        df["year_valid"],
        (df["year"] - 1950) / (current_year - 1950),
        0
    )
    
    # Impact score combining citations and recency
    max_citations = df["citation_count"].max() if len(df) > 0 else 1
    df["normalized_citations"] = df["citation_count"] / max_citations if max_citations > 0 else 0
    df["impact_score"] = (0.7 * df["normalized_citations"] + 0.3 * df["recency_score"])
    
    # Text processing
    df["title_length"] = df["title"].str.len()
    df["has_abstract"] = df["abstract"].str.len() > 50
    df["abstract_length"] = df["abstract"].str.len()
    
    # Author processing
    df["author_count"] = df["author_count"].fillna(0).astype(int)
    df["is_single_author"] = df["author_count"] == 1
    df["is_collaboration"] = df["author_count"] > 3
    
    # Field of study processing
    if "fields_of_study" in df.columns:
        df["field_count"] = df["fields_of_study"].apply(lambda x: len(x) if isinstance(x, list) else 0)
        df["is_interdisciplinary"] = df["field_count"] > 2
    
    # Venue processing
    df["has_venue"] = df["venue"].notna() & (df["venue"] != "")
    
    print(f"Processing completed. Added {len([c for c in df.columns if c not in papers_df.columns])} new features.")
    
    return df

def extract_keywords_from_abstracts(abstracts: List[str], min_length: int = 4, 
                                  top_n: int = 50) -> Dict[str, int]:
    """
    Extract keywords from paper abstracts.
    
    Args:
        abstracts: List of abstract strings
        min_length: Minimum word length to consider
        top_n: Number of top keywords to return
        
    Returns:
        Dictionary of keywords and their frequencies
    """
    # Combine all abstracts
    text = " ".join([abs for abs in abstracts if abs and len(abs) > 50])
    
    if not text:
        return {}
    
    # Simple keyword extraction (could be enhanced with NLP libraries)
    text = text.lower()
    
    # Remove common academic stopwords
    stopwords = {
        "the", "and", "or", "but", "in", "on", "at", "to", "for", "of", "with", 
        "by", "from", "this", "that", "these", "those", "we", "our", "they",
        "paper", "study", "research", "method", "approach", "results", "conclusion",
        "analysis", "data", "using", "based", "show", "shows", "present", "propose"
    }
    
    # Extract words
    words = re.findall(r'\b[a-zA-Z]{' + str(min_length) + ',}\b', text)
    words = [word for word in words if word not in stopwords]
    
    # Count frequencies
    keyword_counts = Counter(words)
    
    return dict(keyword_counts.most_common(top_n))

def create_citation_network(papers_df: pd.DataFrame, citations_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create a citation network from papers and citations data.
    
    Args:
        papers_df: DataFrame with paper information
        citations_df: DataFrame with citation relationships
        
    Returns:
        DataFrame representing citation network edges
    """
    network_edges = []
    
    # Create paper ID to title mapping
    id_to_title = dict(zip(papers_df["paper_id"], papers_df["title"]))
    
    for _, citation in citations_df.iterrows():
        citing_id = citation.get("citing_paper_id")
        cited_id = papers_df.iloc[0]["paper_id"] if len(papers_df) > 0 else None  # Assuming we're analyzing citations to papers in our dataset
        
        if citing_id and cited_id:
            edge = {
                "source": cited_id,
                "target": citing_id,
                "source_title": id_to_title.get(cited_id, "Unknown"),
                "target_title": citation.get("citing_title", "Unknown"),
                "citation_year": citation.get("citing_year"),
                "context_count": citation.get("context_count", 0),
                "weight": 1 + citation.get("context_count", 0)  # Weight by context richness
            }
            network_edges.append(edge)
    
    return pd.DataFrame(network_edges)

def generate_research_summary(papers_df: pd.DataFrame, field_name: str = "research area") -> Dict[str, Any]:
    """
    Generate a comprehensive summary of research data.
    
    Args:
        papers_df: DataFrame with processed paper data
        field_name: Name of the research field being analyzed
        
    Returns:
        Dictionary with comprehensive research summary
    """
    if papers_df.empty:
        return {}
    
    # Basic statistics
    total_papers = len(papers_df)
    year_range = (papers_df["year"].min(), papers_df["year"].max()) if "year" in papers_df.columns else None
    
    # CitedBy analysis
    citation_stats = {
        "total_citations": papers_df["citation_count"].sum(),
        "avg_citations": papers_df["citation_count"].mean(),
        "median_citations": papers_df["citation_count"].median(),
        "max_citations": papers_df["citation_count"].max(),
        "papers_with_citations": (papers_df["citation_count"] > 0).sum(),
    }
    
    # Influence analysis
    if "influential_citation_count" in papers_df.columns:
        influence_stats = {
            "total_influential_citations": papers_df["influential_citation_count"].sum(),
            "avg_influence_ratio": papers_df["influence_ratio"].mean() if "influence_ratio" in papers_df.columns else 0,
            "highly_influential_papers": (papers_df["influential_citation_count"] >= 10).sum()
        }
    else:
        influence_stats = {}
    
    # Temporal analysis
    if "year" in papers_df.columns:
        recent_papers = papers_df[papers_df["year"] >= 2020] if papers_df["year"].max() >= 2020 else pd.DataFrame()
        temporal_stats = {
            "recent_papers_count": len(recent_papers),
            "recent_growth_rate": len(recent_papers) / total_papers if total_papers > 0 else 0,
            "peak_year": papers_df["year"].value_counts().idxmax() if "year" in papers_df.columns else None
        }
    else:
        temporal_stats = {}
    
    # Collaboration analysis
    if "author_count" in papers_df.columns:
        collab_stats = {
            "avg_authors_per_paper": papers_df["author_count"].mean(),
            "single_author_papers": (papers_df["author_count"] == 1).sum(),
            "large_collaborations": (papers_df["author_count"] >= 5).sum(),
            "collaboration_rate": ((papers_df["author_count"] > 1).sum() / total_papers) if total_papers > 0 else 0
        }
    else:
        collab_stats = {}
    
    # Top papers
    top_papers = papers_df.nlargest(5, "citation_count")[["title", "year", "citation_count"]].to_dict("records") if not papers_df.empty else []
    
    # Venue analysis
    venue_stats = {}
    if "venue" in papers_df.columns:
        top_venues = papers_df["venue"].value_counts().head(10).to_dict()
        venue_stats = {"top_venues": top_venues}
    
    summary = {
        "field_name": field_name,
        "overview": {
            "total_papers": total_papers,
            "year_range": year_range,
            **citation_stats
        },
        "influence_metrics": influence_stats,
        "temporal_trends": temporal_stats,
        "collaboration_patterns": collab_stats,
        "venue_distribution": venue_stats,
        "top_papers": top_papers,
        "generated_at": pd.Timestamp.now().isoformat()
    }
    
    return summary

# Example: Process transformer papers data
if 'transformer_papers' in locals() and len(transformer_papers) > 0:
    print("Example 5: Advanced data processing...")
    
    # Process the data
    processed_papers = process_semantic_scholar_data(transformer_papers)
    
    print(f"\nProcessed paper statistics:")
    print(f"- Papers with valid years: {processed_papers['year_valid'].sum()}")
    print(f"- Papers with abstracts: {processed_papers['has_abstract'].sum()}")
    print(f"- Average impact score: {processed_papers['impact_score'].mean():.3f}")
    print(f"- Collaboration rate: {(processed_papers['author_count'] > 1).mean():.1%}")
    
    # Extract keywords from abstracts
    abstracts = processed_papers["abstract"].tolist()
    keywords = extract_keywords_from_abstracts(abstracts, top_n=20)
    print(f"\nTop keywords in abstracts:")
    for keyword, count in list(keywords.items())[:10]:
        print(f"- {keyword}: {count}")
    
    # Generate research summary
    summary = generate_research_summary(processed_papers, "Transformer Research")
    print(f"\nResearch Summary:")
    print(f"- Total papers: {summary['overview']['total_papers']}")
    print(f"- CitedBy range: {summary['overview']['year_range']}")
    print(f"- Average citations: {summary['overview']['avg_citations']:.1f}")
else:
    print("No transformer papers available for processing. Run the paper search cell first.")

## 9. Data Export and Integration

In [ ]:
import os
from datetime import datetime

def export_semantic_scholar_data(datasets: Dict[str, pd.DataFrame], 
                                base_name: str = "semantic_scholar") -> Dict[str, Dict[str, str]]:
    """
    Export multiple DataFrames to various formats.
    
    Args:
        datasets: Dictionary mapping dataset names to DataFrames
        base_name: Base filename for exports
        
    Returns:
        Dictionary mapping dataset names to exported file paths
    """
    # Create data directory
    data_dir = "/home/dnhoa/IAAIR/data"
    os.makedirs(data_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    exported_files = {}
    
    for dataset_name, df in datasets.items():
        if df.empty:
            print(f"Skipping empty dataset: {dataset_name}")
            continue
            
        filename_base = f"{base_name}_{dataset_name}_{timestamp}"
        dataset_files = {}
        
        # Export to CSV
        csv_path = os.path.join(data_dir, f"{filename_base}.csv")
        df.to_csv(csv_path, index=False)
        dataset_files["csv"] = csv_path
        print(f"✅ Exported {dataset_name} to CSV: {csv_path}")
        
        # Export to JSON
        json_path = os.path.join(data_dir, f"{filename_base}.json")
        df.to_json(json_path, orient="records", indent=2)
        dataset_files["json"] = json_path
        print(f"✅ Exported {dataset_name} to JSON: {json_path}")
        
        # Export to Excel (if possible)
        try:
            excel_path = os.path.join(data_dir, f"{filename_base}.xlsx")
            df.to_excel(excel_path, index=False, engine='openpyxl')
            dataset_files["excel"] = excel_path
            print(f"✅ Exported {dataset_name} to Excel: {excel_path}")
        except ImportError:
            print(f"⚠️  Excel export skipped for {dataset_name} (install openpyxl)")
        
        exported_files[dataset_name] = dataset_files
    
    return exported_files

def create_knowledge_fabric_schema(papers_df: pd.DataFrame) -> Dict[str, Any]:
    """
    Create schema mapping for Knowledge Fabric integration.
    
    Args:
        papers_df: Processed papers DataFrame
        
    Returns:
        Schema dictionary for Knowledge Fabric
    """
    # Map Semantic Scholar fields to Knowledge Fabric schema
    schema = {
        "data_source": "semantic_scholar",
        "entity_types": {
            "documents": {
                "id_field": "paper_id",
                "title_field": "title",
                "abstract_field": "abstract",
                "year_field": "year",
                "citation_count_field": "citation_count",
                "authors_field": "authors",
                "venue_field": "venue"
            },
            "authors": {
                "id_field": "author_id",
                "name_field": "name",
                "paper_count_field": "paper_count",
                "citation_count_field": "citation_count",
                "h_index_field": "h_index"
            }
        },
        "relationships": {
            "authorship": {
                "source": "authors",
                "target": "documents",
                "properties": ["position", "is_corresponding"]
            },
            "citation": {
                "source": "documents",
                "target": "documents", 
                "properties": ["citation_contexts", "influential"]
            }
        },
        "field_mappings": {
            # Map S2 fields to standard names
            "paperId": "document_id",
            "authorId": "author_id",
            "citationCount": "citation_count",
            "influentialCitationCount": "influential_citation_count",
            "isOpenAccess": "is_open_access"
        },
        "sample_data": papers_df.head(3).to_dict("records") if not papers_df.empty else []
    }
    
    return schema

def export_for_knowledge_fabric(papers_df: pd.DataFrame, authors_df: pd.DataFrame = None,
                               citations_df: pd.DataFrame = None) -> str:
    """
    Export data in Knowledge Fabric compatible format.
    
    Args:
        papers_df: Papers DataFrame
        authors_df: Authors DataFrame (optional)
        citations_df: Citations DataFrame (optional)
        
    Returns:
        Path to exported JSON file
    """
    data_dir = "/home/dnhoa/IAAIR/data"
    os.makedirs(data_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Create Knowledge Fabric compatible structure
    kg_data = {
        "metadata": {
            "source": "semantic_scholar",
            "exported_at": datetime.now().isoformat(),
            "schema_version": "1.0",
            "total_papers": len(papers_df) if not papers_df.empty else 0,
            "total_authors": len(authors_df) if authors_df is not None and not authors_df.empty else 0,
            "total_citations": len(citations_df) if citations_df is not None and not citations_df.empty else 0
        },
        "documents": papers_df.to_dict("records") if not papers_df.empty else [],
        "authors": authors_df.to_dict("records") if authors_df is not None and not authors_df.empty else [],
        "citations": citations_df.to_dict("records") if citations_df is not None and not citations_df.empty else [],
        "schema": create_knowledge_fabric_schema(papers_df)
    }
    
    # Export to JSON
    kg_file = os.path.join(data_dir, f"knowledge_fabric_import_{timestamp}.json")
    with open(kg_file, 'w', encoding='utf-8') as f:
        json.dump(kg_data, f, indent=2, default=str)
    
    print(f"✅ Exported Knowledge Fabric data: {kg_file}")
    print(f"   - Papers: {len(kg_data['documents'])}")
    print(f"   - Authors: {len(kg_data['authors'])}")
    print(f"   - Citations: {len(kg_data['citations'])}")
    
    return kg_file

def create_data_summary_report(exported_files: Dict[str, Dict[str, str]]) -> pd.DataFrame:
    """
    Create a summary report of all exported data.
    
    Args:
        exported_files: Dictionary of exported file paths
        
    Returns:
        Summary DataFrame
    """
    summary_data = []
    
    for dataset_name, files in exported_files.items():
        for format_type, filepath in files.items():
            # Get file size
            file_size = os.path.getsize(filepath) if os.path.exists(filepath) else 0
            file_size_mb = round(file_size / (1024 * 1024), 2)
            
            summary = {
                "dataset": dataset_name,
                "format": format_type,
                "filepath": filepath,
                "size_mb": file_size_mb,
                "exported_at": datetime.now().isoformat()
            }
            summary_data.append(summary)
    
    return pd.DataFrame(summary_data)

# Example: Export all collected data
datasets_to_export = {}

# Collect available datasets
if 'transformer_papers' in locals():
    datasets_to_export['transformer_papers'] = transformer_papers
if 'processed_papers' in locals():
    datasets_to_export['processed_papers'] = processed_papers
if 'ai_researchers' in locals():
    datasets_to_export['ai_researchers'] = ai_researchers
if 'citations' in locals():
    datasets_to_export['citations'] = citations
if 'recommendations' in locals():
    datasets_to_export['recommendations'] = recommendations

if datasets_to_export:
    print("Exporting Semantic Scholar datasets...")
    
    # Export to multiple formats
    exported_files = export_semantic_scholar_data(datasets_to_export)
    
    # Export for Knowledge Fabric
    papers_data = datasets_to_export.get('processed_papers', datasets_to_export.get('transformer_papers'))
    authors_data = datasets_to_export.get('ai_researchers')
    citations_data = datasets_to_export.get('citations')
    
    if papers_data is not None and not papers_data.empty:
        kg_file = export_for_knowledge_fabric(papers_data, authors_data, citations_data)
    
    # Create summary report
    summary_report = create_data_summary_report(exported_files)
    
    print(f"\n=== Export Summary ===")
    print(summary_report.to_string(index=False))
    
    # Export summary report
    summary_path = os.path.join("/home/dnhoa/IAAIR/data", f"export_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
    summary_report.to_csv(summary_path, index=False)
    print(f"\nExport summary saved: {summary_path}")
    
else:
    print("No datasets available for export. Run the data collection cells first.")

## 10. Visualization and Analysis

Quick visualizations to understand the data patterns:

In [ ]:
def create_visualizations(papers_df: pd.DataFrame):
    """Create basic visualizations for paper data analysis"""
    
    if papers_df.empty:
        print("No data available for visualization")
        return
    
    try:
        # Set up the plotting style
        plt.style.use('default')
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Semantic Scholar Data Analysis', fontsize=16, fontweight='bold')
        
        # 1. Papers by year
        if 'year' in papers_df.columns:
            valid_years = papers_df[papers_df['year'].notna() & (papers_df['year'] >= 1990)]
            if not valid_years.empty:
                yearly_counts = valid_years['year'].value_counts().sort_index()
                axes[0, 0].plot(yearly_counts.index, yearly_counts.values, marker='o', linewidth=2, markersize=4)
                axes[0, 0].set_title('Publications by Year', fontweight='bold')
                axes[0, 0].set_xlabel('Year')
                axes[0, 0].set_ylabel('Number of Papers')
                axes[0, 0].grid(True, alpha=0.3)
        
        # 2. CitedBy distribution
        if 'citation_count' in papers_df.columns:
            citations = papers_df['citation_count'][papers_df['citation_count'] >= 0]
            if not citations.empty:
                # Use log scale for better visualization
                log_citations = np.log1p(citations)  # log(1 + x) to handle 0 citations
                axes[0, 1].hist(log_citations, bins=30, alpha=0.7, edgecolor='black')
                axes[0, 1].set_title('CitedBy Distribution (Log Scale)', fontweight='bold')
                axes[0, 1].set_xlabel('Log(1 + Citations)')
                axes[0, 1].set_ylabel('Number of Papers')
                axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Collaboration patterns
        if 'author_count' in papers_df.columns:
            author_counts = papers_df['author_count'][papers_df['author_count'] > 0]
            if not author_counts.empty:
                collaboration_dist = author_counts.value_counts().sort_index()[:10]  # Top 10 author counts
                axes[1, 0].bar(collaboration_dist.index, collaboration_dist.values, alpha=0.7, edgecolor='black')
                axes[1, 0].set_title('Collaboration Patterns', fontweight='bold')
                axes[1, 0].set_xlabel('Number of Authors')
                axes[1, 0].set_ylabel('Number of Papers')
                axes[1, 0].grid(True, alpha=0.3)
        
        # 4. Top venues
        if 'venue' in papers_df.columns:
            venues = papers_df['venue'].dropna()
            if not venues.empty and len(venues) > 0:
                top_venues = venues.value_counts().head(10)
                axes[1, 1].barh(range(len(top_venues)), top_venues.values, alpha=0.7, edgecolor='black')
                axes[1, 1].set_yticks(range(len(top_venues)))
                axes[1, 1].set_yticklabels([v[:30] + '...' if len(v) > 30 else v for v in top_venues.index], fontsize=9)
                axes[1, 1].set_title('Top Publication Venues', fontweight='bold')
                axes[1, 1].set_xlabel('Number of Papers')
                axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Visualization error: {e}")
        print("Note: Make sure matplotlib and seaborn are installed for visualizations")

def print_data_insights(papers_df: pd.DataFrame):
    """Print key insights from the data"""
    
    if papers_df.empty:
        print("No data available for insights")
        return
    
    print("=== Key Data Insights ===\n")
    
    # Basic stats
    print(f"📊 Dataset Overview:")
    print(f"   Total papers: {len(papers_df):,}")
    
    if 'year' in papers_df.columns:
        valid_years = papers_df[papers_df['year'].notna()]
        if not valid_years.empty:
            print(f"   Year range: {valid_years['year'].min():.0f} - {valid_years['year'].max():.0f}")
            recent_papers = valid_years[valid_years['year'] >= 2020]
            print(f"   Recent papers (2020+): {len(recent_papers):,} ({len(recent_papers)/len(valid_years):.1%})")
    
    # CitedBy insights
    if 'citation_count' in papers_df.columns:
        citations = papers_df['citation_count'][papers_df['citation_count'] >= 0]
        if not citations.empty:
            print(f"\n📈 CitedBy Metrics:")
            print(f"   Total citations: {citations.sum():,}")
            print(f"   Average citations: {citations.mean():.1f}")
            print(f"   Median citations: {citations.median():.1f}")
            print(f"   Most cited paper: {citations.max():,} citations")
            highly_cited = citations[citations >= 100]
            print(f"   Highly cited papers (100+ citations): {len(highly_cited):,} ({len(highly_cited)/len(citations):.1%})")
    
    # Collaboration insights
    if 'author_count' in papers_df.columns:
        authors = papers_df['author_count'][papers_df['author_count'] > 0]
        if not authors.empty:
            print(f"\n🤝 Collaboration Patterns:")
            print(f"   Average authors per paper: {authors.mean():.1f}")
            single_author = authors[authors == 1]
            print(f"   Single-author papers: {len(single_author):,} ({len(single_author)/len(authors):.1%})")
            large_collab = authors[authors >= 5]
            print(f"   Large collaborations (5+ authors): {len(large_collab):,} ({len(large_collab)/len(authors):.1%})")
    
    # Open access insights
    if 'is_open_access' in papers_df.columns:
        oa_data = papers_df['is_open_access']
        oa_count = oa_data.sum() if oa_data.dtype == bool else (oa_data == True).sum()
        print(f"\n🔓 Open Access:")
        print(f"   Open access papers: {oa_count:,} ({oa_count/len(papers_df):.1%})")
    
    # Top papers
    if 'citation_count' in papers_df.columns and 'title' in papers_df.columns:
        top_papers = papers_df.nlargest(3, 'citation_count')
        print(f"\n🏆 Most CitedBy Papers:")
        for i, (_, paper) in enumerate(top_papers.iterrows(), 1):
            title = paper['title'][:60] + "..." if len(str(paper['title'])) > 60 else paper['title']
            citations = paper['citation_count']
            year = f" ({paper['year']:.0f})" if 'year' in papers_df.columns and pd.notna(paper['year']) else ""
            print(f"   {i}. {title}{year} - {citations:,} citations")

# Generate visualizations and insights if we have data
if 'transformer_papers' in locals() and not transformer_papers.empty:
    print("Example 6: Data Visualization and Insights")
    print_data_insights(transformer_papers)
    print("\nGenerating visualizations...")
    create_visualizations(transformer_papers)
elif 'processed_papers' in locals() and not processed_papers.empty:
    print("Example 6: Data Visualization and Insights")
    print_data_insights(processed_papers)
    print("\nGenerating visualizations...")
    create_visualizations(processed_papers)
else:
    print("No paper data available for visualization. Run the data collection cells first.")

## 11. Next Steps and Integration

### Key Features of Semantic Scholar API:
- **Rich CitedBy Data**: Full citation contexts and influential citation metrics
- **AI-Powered Recommendations**: Advanced paper recommendation system
- **Author Networks**: Comprehensive author profiles and collaboration data
- **Embeddings**: Paper embeddings for semantic similarity
- **TLDR Generation**: AI-generated paper summaries

### Integration with Knowledge Fabric:
1. **Document Ingestion**: Use the processed papers data for document entities
2. **Author Networks**: Leverage author collaboration data for network analysis
3. **CitedBy Graphs**: Build citation networks with context information
4. **Semantic Search**: Use paper embeddings for similarity search
5. **Attribution**: Use citation contexts for evidence attribution

### Best Practices:
1. **API Key**: Get an API key for higher rate limits (1000 req/5min)
2. **Batch Processing**: Use batch endpoints for efficiency
3. **CitedBy Context**: Leverage rich citation context data
4. **Influential Metrics**: Use influential citation counts for impact assessment
5. **Data Quality**: Filter and validate data before processing

### Complementary to OpenAlex:
- **OpenAlex**: Broader coverage, completely open data
- **Semantic Scholar**: Richer features, AI-powered insights, citation contexts
- **Combined Use**: Use both APIs for comprehensive coverage

### Resources:
- [Semantic Scholar API Documentation](https://api.semanticscholar.org/)
- [Academic Graph Dataset](https://www.semanticscholar.org/product/api#academic-graph-api)
- [API Key Registration](https://www.semanticscholar.org/product/api#api-key)